In [2]:
import cv2
# import time
import numpy as np
# import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import ipywidgets as widgets

In [3]:
# 自定义的一些常量范围和函数，还有库
#import move
import readygo

# 定义红色的HSV范围
# 红色在 HSV 中有两个不连续区间，故定义两个区域
lower_red1 = np.array([0, 80, 80])
upper_red1 = np.array([15, 255, 255])
lower_red2 = np.array([165, 80, 80])
upper_red2 = np.array([180, 255, 255])

# 展示图片函数
def imshow(img):
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    # plt.axis('off')  # 关闭坐标轴
    plt.show()

## 读取中心点判断红色变色

In [25]:
cap = cv2.VideoCapture('video/red-green.mp4')  # CAP_DSHOW 可避免某些 Windows 卡顿
if not cap.isOpened():
    print("无法打开摄像头")
    exit()
read_or_not = 0
try:
    while True:
        ret, frame = cap.read()
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        if not ret:
            print("警告：无法从摄像头读取帧")
            break

        if(read_or_not == 0):
            result = readygo.detect_red2(hsv)
            read_or_not = 1
            print(f'色块位置描述{result}')
        if(hsv[result[1]+result[3]//2,result[0]+result[2]//2,2]<70):
            # readygo.move(0,5,0)
            print('have been changed')
            break

except Exception as e:
    print(f"发生异常：{e}")
finally:
    # 确保资源释放
    cap.release()

色块位置描述(244, 211, 9, 6)
have been changed


## 监视红色像素块探测

In [32]:
cap = cv2.VideoCapture('video/red-green.mp4')  # CAP_DSHOW 可避免某些 Windows 卡顿
if not cap.isOpened():
    print("无法打开摄像头")
    exit()
try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("警告：无法从摄像头读取帧")
            break

        result = readygo.detect_red(frame)
        if len(result) <2:
            print('灯变化')
            break
except Exception as e:
    print(f"发生异常：{e}")
finally:
    # 确保资源释放
    cap.release()

灯变化


## 监视视频

In [1]:
cap = cv2.VideoCapture(2)#'video/red-green.mp4')  # CAP_DSHOW 可避免某些 Windows 卡顿
if not cap.isOpened():
    print("无法打开摄像头")
    exit()
i = 0
try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("警告：无法从摄像头读取帧")
            break
        cv2.imshow('A',frame)
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
         # 按 s 保存当前帧，按 q 退出
        key = cv2.waitKey(10) & 0xFF
        if key == ord('s'):
            cv2.imwrite(f'D:picture\{i}.png', frame)
            i += 1
        elif key == ord('q'):
            break
except Exception as e:
    print(f"发生异常：{e}")
finally:
    # 确保资源释放
    cap.release()
    cv2.destroyAllWindows()

NameError: name 'cv2' is not defined

# 滑块控制示例

In [ ]:
# -------------图像预处理------------------------
# 读取图片 (BGK)
image = cv2.imread("picture/0.png")
if image is None:
    raise ValueError("无法读取图片，请确认路径是否正确")

# ---------------创建控制端，输出端和回调函数----------------------
# 回调函数
def on_value_change(change):
    LOWEST = change['new']
    mask0,mask1,draw = readygo.detect_red_info(image, LOWEST)
    # 输出的内容
    with output:
        clear_output(wait=True)# 清空前面的输出内容
        # imshow(draw)
        # imshow(mask)
        # plt.figure(figsize=(14,9))
        plt.figure(figsize=(10,7))
        plt.subplot(1, 2, 1)
        plt.imshow(cv2.cvtColor(mask0, cv2.COLOR_BGR2RGB))
        plt.title("mask_origin")
        plt.subplot(1, 2, 2)
        plt.imshow(cv2.cvtColor(draw, cv2.COLOR_BGR2RGB))
        plt.title("draw")
        plt.show()
# 控件(输入和输出)
output = widgets.Output()# 输出端
slider = widgets.IntSlider(value=100, min=0, max=100, step=1, description='LOWEST:')# 控制端
slider.observe(on_value_change, names='value')# 绑定控制端和回调函数
display(slider, output)# 展示控制端和输出端
on_value_change({'new': slider.value})# 初次触发

## 变看图像变追踪

In [42]:
cap = cv2.VideoCapture('video/red-green.mp4')  # CAP_DSHOW 可避免某些 Windows 卡顿
if not cap.isOpened():
    print("无法打开摄像头")
    exit()
# ---------------------------连续五帧为空即可-------------------------------------
j = 0
try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("警告：无法从摄像头读取帧")
            break
        result = readygo.detect_red(frame);print(result)
        if len(result)>2:
            cv2.rectangle(frame, (result[0], result[1]), (result[0]+result[2], result[1] + result[3]), (0, 255, 0), -1)
        else :
            j+=1
        cv2.imshow('A',frame)
        # 按 s 保存当前帧，按 q 退出
        key = cv2.waitKey(10) & 0xFF
        if key == ord('q'):
            break
        if(j>=5):
            print(j)
            print('灯变换')
except Exception as e:
    print(f"发生异常：{e}")
finally:
    # 确保资源释放
    cap.release()
    cv2.destroyAllWindows()

(244, 211, 9, 6)
(244, 211, 9, 6)
(243, 211, 10, 7)
(243, 211, 10, 6)
(243, 211, 10, 6)
(243, 211, 10, 6)
(244, 211, 9, 6)
(244, 211, 9, 6)
(244, 211, 9, 6)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211, 9, 7)
(244, 211,

## 控件实现

In [5]:
# -----------------------库导入--------------------------
import readygo
import cv2
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# ----------------------图像与显示函数---------------------
image = cv2.imread("picture/0.png")
if image is None:
    raise ValueError("无法读取图片，请确认路径是否正确")

def imshow(img):
    plt.figure(figsize=(5, 5))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

# ----------------------控件定义------------------------
# 输出窗口
output = widgets.Output()

# LOWEST 使用单值滑块
slider_lowest = widgets.IntSlider(value=100, min=0, max=100, step=1, description='LOWEST:', continuous_update=False)
# 红色 HSV 范围使用区间滑块
slider_red1_h = widgets.IntRangeSlider(value=[0, 10], min=0, max=180, step=1, description='Red1 H:', continuous_update=False)
slider_red1_s = widgets.IntRangeSlider(value=[60, 255], min=0, max=255, step=1, description='Red1 S:', continuous_update=False)
slider_red1_v = widgets.IntRangeSlider(value=[60, 255], min=0, max=255, step=1, description='Red1 V:', continuous_update=False)

slider_red2_h = widgets.IntRangeSlider(value=[165, 180], min=0, max=180, step=1, description='Red2 H:', continuous_update=False)
slider_red2_s = widgets.IntRangeSlider(value=[60, 255], min=0, max=255, step=1, description='Red2 S:', continuous_update=False)
slider_red2_v = widgets.IntRangeSlider(value=[60, 255], min=0, max=255, step=1, description='Red2 V:', continuous_update=False)
# 分组排列
group_red1 = widgets.HBox([slider_red1_h, slider_red2_h])
group_red2 = widgets.HBox([slider_red1_s, slider_red2_s])
group_red3 = widgets.HBox([slider_red1_v, slider_red2_v])

# ----------------------回调函数------------------------
def on_value_change(change=None):
    # 获取 LOWEST 阈值
    LOWEST = slider_lowest.value

    # 获取红色范围
    lower_red1 = np.array([slider_red1_h.value[0], slider_red1_s.value[0], slider_red1_v.value[0]])
    upper_red1 = np.array([slider_red1_h.value[1], slider_red1_s.value[1], slider_red1_v.value[1]])

    lower_red2 = np.array([slider_red2_h.value[0], slider_red2_s.value[0], slider_red2_v.value[0]])
    upper_red2 = np.array([slider_red2_h.value[1], slider_red2_s.value[1], slider_red2_v.value[1]])

    # 调用处理函数
    mask0, mask1, draw = readygo.detect_red_info(image, LOWEST, lower_red1, upper_red1, lower_red2, upper_red2)

    # 显示图像
    with output:
        clear_output(wait=True)
        plt.figure(figsize=(10, 7))
        plt.subplot(1, 2, 1)
        plt.imshow(cv2.cvtColor(draw, cv2.COLOR_BGR2RGB))
        plt.title("Result")
        plt.subplot(1, 2, 2)
        plt.imshow(mask0, cmap='gray')
        plt.title("Mask")
        plt.show()

# 绑定回调
sliders = [slider_lowest, slider_red1_h, slider_red1_s, slider_red1_v,
           slider_red2_h, slider_red2_s, slider_red2_v]
for s in sliders:
    s.observe(on_value_change, names='value')


In [6]:
# ----------------------界面展示------------------------
controls = widgets.VBox([slider_lowest, group_red1, group_red2, group_red3])
display(controls, output)

# ----------------------首次触发------------------------
on_value_change()

Output()

In [22]:
btn_close = widgets.Button(description='关闭', button_style='danger')

def on_close_clicked(b):
    with output:
        clear_output(wait=True)
    # 禁用所有控件
    for slider in [slider_lowest, slider_red1_h, slider_red1_s, slider_red1_v,
                   slider_red2_h, slider_red2_s, slider_red2_v]:
        slider.disabled = True
    btn_close.disabled = True

btn_close.on_click(on_close_clicked)

display(btn_close)


Button(button_style='danger', description='关闭', style=ButtonStyle())

In [ ]:
import cv2
import matplotlib.pyplot as plt
import threading
import time
from IPython.display import display, clear_output
import ipywidgets as widgets

# 加载视频
cap = cv2.VideoCapture("video/red-green.mp4")
if not cap.isOpened():
    raise ValueError("无法打开视频文件")

output = widgets.Output()
display(output)

def video_loop():
    while True:
        ret, frame = cap.read()
        if not ret:
            cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
            continue

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        with output:
            clear_output(wait=True)
            plt.figure(figsize=(6, 4))
            plt.imshow(frame_rgb)
            plt.axis('off')
            plt.title("Video Frame")
            plt.show()

        time.sleep(0.03)

thread = threading.Thread(target=video_loop, daemon=True)
thread.start()
